In [17]:
import torch
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')



In [4]:
# Check if the notebook is running in Google Colab
def is_running_in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

root = ""

# Adjust the path as per Google Drive directory structure
if is_running_in_colab():
    from google.colab import drive
    drive.mount('/content/drive')
    root = "/content/drive/My Drive/"
else:
    print("The notebook is running in a local environment.")
    



The notebook is running in a local environment.


In [5]:
# load the data
ratings_path = root + 'data/ratings.dat'
movies_path = root + 'data/movies.dat'

ratings = pd.read_csv(ratings_path, sep='::', engine='python', header=None)
ratings.columns = ['UserID', 'MovieID', 'Rating', 'Timestamp']

movies = pd.read_csv(movies_path, sep='::', engine='python', encoding="ISO-8859-1", header=None)
movies.columns = ['MovieID', 'Title', 'Genres']

In [6]:
# define user and movie rating matrix
item_feature_matrix = ratings.pivot_table(index='UserID', columns='MovieID', values='Rating')

# get mean ratings per row (user)
mean_ratings = np.array(item_feature_matrix.mean(axis=1, skipna=True)).reshape(-1, 1)

# normalize ratings by subtracting mean rating per user
normalized_ratings = item_feature_matrix - mean_ratings



In [14]:
# get intersection of two tensors
def intersect1d_torch(tensor1, tensor2):
    # Find unique elements in each tensor
    unique1 = torch.unique(tensor1)
    unique2 = torch.unique(tensor2)

    # Find common elements
    common = unique1[torch.isin(unique1, unique2)]

    return common

# compute similarity matrix by tensors
def compute_similarity_matrix_tensor(rating, min_common_users = 3, top_n = 10):
    min_common_users = min_common_users
    top_n = top_n

    # Check if CUDA (GPU support) is available
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(device)

    # Convert DataFrame to tensors
    rating_0_tensor = torch.tensor(rating.fillna(0).values, dtype=torch.float32)
    rating_binary_tensor = (rating_0_tensor != 0).float()

    # Compute common ratings count
    common_ratings_count = torch.mm(rating_binary_tensor.T, rating_binary_tensor)

    # Prepare the similarity matrix
    num_movies = rating_0_tensor.shape[1]
    #similarity_matrix = torch.full((num_movies, num_movies), float('nan'))
    similarity_matrix = torch.full((num_movies, num_movies), float('nan')).to(device)


    for i in range(num_movies):
        for j in range(i + 1, num_movies):
            if common_ratings_count[i, j] >= min_common_users:
                # Filter ratings to include only common users
                user_0 = torch.where(rating_binary_tensor[:, i] == 1)[0]
                user_1 = torch.where(rating_binary_tensor[:, j] == 1)[0]
                common_users = intersect1d_torch(user_0, user_1)
                ratings_i = rating_0_tensor[common_users, i]
                ratings_j = rating_0_tensor[common_users, j]

                # Compute dot product and norms for the filtered ratings
                dot_product = torch.dot(ratings_i, ratings_j)
                norm_i = torch.norm(ratings_i)
                norm_j = torch.norm(ratings_j)

                if norm_i > 0 and norm_j > 0:
                    similarity = 0.5 + 0.5 * (dot_product / (norm_i * norm_j))
                else:
                    similarity = 0

                similarity_matrix[i, j] = similarity
                similarity_matrix[j, i] = similarity

    # Convert the similarity matrix to a DataFrame
    similarity_matrix_df = pd.DataFrame(similarity_matrix.cpu().numpy(),
                                       index=rating.columns,
                                       columns=rating.columns)
    
    # Filter the top_n most similar movies for each movie (row)
    if top_n is not None:
        for i in range(similarity_matrix_df.shape[0]):
            row_indices = np.argsort(-similarity_matrix_df.iloc[i])[:top_n]
            mask = np.ones(similarity_matrix_df.shape[1], dtype=bool)
            mask[row_indices] = False
            similarity_matrix_df.iloc[i, mask] = np.nan

    return similarity_matrix_df


In [18]:
specified_movies = [1, 10, 100]
r_n_matrix = normalized_ratings.iloc[:500, :100]
reduced_similarity_matrix = compute_similarity_matrix_tensor(r_n_matrix, min_common_users=3, top_n=30)
#print(reduced_similarity_matrix.shape)
#print(reduced_similarity_matrix)
print(reduced_similarity_matrix.loc[specified_movies, specified_movies].round(7))

cpu
MovieID       1         10   100
MovieID                         
1             NaN  0.615281  NaN
10       0.615281       NaN  NaN
100           NaN       NaN  NaN


In [ ]:
specified_movies = [1, 10, 100, 260]
r_n_matrix = normalized_ratings.iloc[:500, :300]
reduced_similarity_matrix = compute_similarity_matrix_tensor(r_n_matrix, 3, 30)
#print(reduced_similarity_matrix.shape)
#print(reduced_similarity_matrix)
print(reduced_similarity_matrix.loc[specified_movies, specified_movies].round(7))

In [55]:
# get the movie titles by movie ids
def get_movie_titles(movie_ids):
    return movies[movies['MovieID'].isin(movie_ids)].reset_index(drop=True)

In [72]:
# load similarity matrix from local npz file
data = np.load('data/similarity_matrix_top30.npz')
similarity_matrix_top30 = data['matrix']

# convert similarity matrix to dataframe
similarity_matrix_top30_df = pd.DataFrame(similarity_matrix_top30,
                                          index=item_feature_matrix.columns,
                                          columns=item_feature_matrix.columns)

print(similarity_matrix_top30_df)

MovieID  1     2     3     4     5     6     7     8     9     10    ...  \
MovieID                                                              ...   
1         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
2         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
4         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
5         NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
...       ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
3948      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3949      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3950      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3951      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3952      NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   

MovieID  39

In [75]:
def recommend_movies(user_rating, s_matrix, n_recommendations = 10):
    """
    Generate movie recommendations based on new user ratings and a sparse similarity matrix.

    Parameters:
    - user_ratings: np.array, user's ratings for movies; 0 indicates the movie hasn't been rated.
    - s_matrix: item-item similarity matrix.
    - n_recommendations: int, the number of recommendations to return.

    Returns:
    - List of movie indices representing the top N recommendations.
    """

    # number of movies
    n_movies = len(user_rating)

    # initial rated movies
    ini_movies = set(np.where(user_rating != 0)[0])
    #print(ini_movies)

    # iterate through each movie and calculate the weighted rating for user
    for l in range(n_movies):
        if user_rating[l] == 0:
            # get the similarity score for moviel l
            l_s_scores = np.nan_to_num(s_matrix.iloc[l])
            
            # get the weighted sum for movie l
            weighted_sum = np.sum(user_rating * l_s_scores)

            # get the normalization factor
            norm_factor = np.sum(l_s_scores * (user_rating != 0))

            # get the weighted average for movie l
            weighted_avg = np.sum(weighted_sum) / norm_factor if norm_factor != 0 else 0
            #print(f'{l}: {weighted_avg}') if norm_factor != 0 and weighted_avg > 4 else None

            #print(l, " ", weighted_sum, " ", norm_factor, " ", weighted_avg)
            user_rating[l] = weighted_avg

    # get the top N recommendations and not in ini_movies
    top_n_plus = np.argsort(user_rating)[-(n_recommendations + len(ini_movies)):][::-1]
    #print(top_n_plus)

    # exclude ratings already exist, and take the first 10 in the remainder
    top_n = np.array([n for n in top_n_plus if n not in ini_movies][:10])
    #print(top_n)
    top_n_id = s_matrix.columns[top_n]
    
    top_n_score = user_rating[top_n]
    #print(top_n_score)
        

    return top_n, top_n_score

In [76]:
# generate a vector 1 x 300, which represents the rating of user 1 to all movies


user_1181_ratings = ratings[ratings['UserID'] == 1181]
user_1181_ratings = user_1181_ratings.set_index('MovieID')['Rating']
user_1181_ratings = user_1181_ratings.reindex(range(1, 3707), fill_value=0.0)

movie_ids, scores = recommend_movies(user_1181_ratings.values, similarity_matrix_top30_df, 10)

get_movie_titles(movie_ids)

,MovieID,Title,Genres
0,95,Broken Arrow (1996),Action|Thriller
1,101,Bottle Rocket (1996),Comedy
2,238,Far From Home: The Adventures of Yellow Dog (1...,Adventure|Children's
3,247,Heavenly Creatures (1994),Drama|Fantasy|Romance|Thriller
4,384,Bad Company (1995),Action
5,554,Trial by Jury (1994),Thriller
6,2535,Earthquake (1974),Action
7,2900,Monkey Shines (1988),Horror|Sci-Fi
8,3179,Angela's Ashes (1999),Drama
9,3399,Sesame Street Presents Follow That Bird (1985),Children's|Comedy


In [54]:
# get the movie titles by movie ids
def get_movie_titles(movie_ids):
    return movies[movies['MovieID'].isin(movie_ids)]

get_movie_titles(movie_ids)

,MovieID,Title,Genres
24,25,Leaving Las Vegas (1995),Drama|Romance
25,26,Othello (1995),Drama
65,66,Lawnmower Man 2: Beyond Cyberspace (1996),Sci-Fi|Thriller
68,69,Friday (1995),Comedy
81,82,Antonia's Line (Antonia) (1995),Drama
82,83,Once Upon a Time... When We Were Colored (1995),Drama
84,85,Angels and Insects (1995),Drama|Romance
85,86,White Squall (1996),Adventure|Drama
86,87,Dunston Checks In (1996),Children's|Comedy
87,88,Black Sheep (1996),Comedy


In [ ]:
# sketch work


rating_0 = normalized_ratings.fillna(0)

# take the first 100 usrs and 500 movies
r_n_matrix = rating_0.iloc[:100, :500]

rating_binary = (r_n_matrix != 0).astype(int)
common_ratings_count = np.array(rating_binary.T.dot(rating_binary)) # fix 1
print(common_ratings_count)

# find users who have rated both movie 0
user_0 = rating_binary[rating_binary.iloc[:, 0] == 1].index - 1 # index starts from 1, fix 2
print(user_0)

# find users who have rated both movie 2
user_2 = rating_binary[rating_binary.iloc[:, 2] == 1].index - 1 # index starts from 1
print(user_2)

# # print user 25's rating for movie 0
# print(r_n_matrix.iloc[25, 0])

# # print user 44's rating for movie 0
# print(r_n_matrix.iloc[44, 0])

# # print user 25's rating for movie 2
# print(r_n_matrix.iloc[25, 2])

# # print user 44's rating for movie 2
# print(r_n_matrix.iloc[44, 2])

# find common users between user_0 and user_2
common_users = np.intersect1d(user_0, user_2) # fix 3
print(common_users)

# find ratings of common users for movie 0
movie_0_ratings = r_n_matrix.iloc[common_users, 0].values # fix 4
movie_0_ratings = r_n_matrix.iloc[common_users, 0].values # fix 4
print(movie_0_ratings)

# find ratings of common users for movie 2
movie_2_ratings = r_n_matrix.iloc[common_users, 2].values # fix 5
print(movie_2_ratings)

# calculate the cosine similarity between movie 0 and movie 2
product = np.dot(movie_0_ratings, movie_2_ratings)
norm_0 = np.linalg.norm(movie_0_ratings)
norm_2 = np.linalg.norm(movie_2_ratings)

score = product / (norm_0 * norm_2)

print(score)
print(0.5 + 0.5 * score)
